# Korean Tokenizer 학습

210319

huggingface의 tokenizers를 이용해서 한글 tokenizer를 학습한다

tokenizer에 special_tokens는 굉장히 중요하다.

하지만 tokenizers로 학습시 special_tokens를 추가하지만

transformers에서 load할 때 반영되지 않는다

그래서 transformers에서 load 후 special_tokens를 추가하고 저장하면 load 가능하다

tokenizers와 transformers 사이의 매끄러운 연결 구현이 huggingface에 잘 안되있다

억지로 되게 만들었다


### 주의!!
BertWordPieceTokenizer 학습 시 한국어는 strip_accents를 False로 해줘야 한다

만약 True일 시 나는 -> 'ㄴ','ㅏ','ㄴ','ㅡ','ㄴ' 로 쪼개져서 처리된다

학습시 False했으므로 load할 때도 False를 꼭 확인해야 한다


- [keep steady](https://keep-steady.tistory.com/37?category=702926)
- [transformers](https://github.com/huggingface/transformers/blob/master/notebooks/02-transformers.ipynb)

# 0. 설치(210319 최신버전)

# 1. mecab 적용

In [1]:
# load korean corpus for tokenizer training
with open('data/train_tokenizer.txt', 'r', encoding='utf-8') as f:
    data = f.read().split('\n')
print(data[:3])

['어릴때보고 지금다시봐도 재밌어요ㅋㅋ', '디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산업이 부러웠는데. 사실 우리나라에서도 그 어려운시절에 끝까지 열정을 지킨 노라노 같은 전통이있어 저와 같은 사람들이 꿈을 꾸고 이뤄나갈 수 있다는 것에 감사합니다.', '폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.']


In [2]:
%%time
# mecab for window는 아래 코드 사용
from konlpy.tag import Mecab  # install mecab for window: https://hong-yp-ml-records.tistory.com/91
mecab_tokenizer = Mecab(dicpath=r"C:\mecab\mecab-ko-dic").morphs
print('mecab check :', mecab_tokenizer('어릴때보고 지금다시봐도 재밌어요ㅋㅋ'))

# 1: '어릴때' -> '어릴, ##때' for generation model
# 2: '어릴때' -> '어릴, 때'   for normal case

# ['어릴', '때', '보', '고', '지금', '다시', '봐도', '재밌', '어요', 'ㅋㅋ']
# ['어릴', '##때', '##보', '##고', '##지금', '##다시', '##봐도', '##재밌', '##어요', '##ㅋㅋ']

for_generation = False # or normal

if for_generation:
    # 1: '어릴때' -> '어릴, ##때' for generation model
    total_morph=[]
    for sentence in data:
        # 문장단위 mecab 적용
        morph_sentence= []
        count = 0
        for token_mecab in mecab_tokenizer(sentence):
            token_mecab_save = token_mecab
            if count > 0:
                token_mecab_save = "##" + token_mecab_save  # 앞에 ##를 부친다
                morph_sentence.append(token_mecab_save)
            else:
                morph_sentence.append(token_mecab_save)
                count += 1
        # 문장단위 저장
        total_morph.append(morph_sentence)

else:
    # 2: '어릴때' -> '어릴, 때'   for normal case
    total_morph=[]
    for sentence in data:
        # 문장단위 mecab 적용
        morph_sentence= mecab_tokenizer(sentence)
        # 문장단위 저장
        total_morph.append(morph_sentence)
                        
print(total_morph[:3])
print(len(total_morph))

# mecab 적용한 데이터 저장
# ex) 1 line: '어릴 때 보 고 지금 다시 봐도 재밌 어요 ㅋㅋ'
with open('data/after_mecab.txt', 'w', encoding='utf-8') as f:
    for line in total_morph:
        f.write(' '.join(line)+'\n')

mecab check : ['어릴', '때', '보', '고', '지금', '다시', '봐도', '재밌', '어요', 'ㅋㅋ']
[['어릴', '때', '보', '고', '지금', '다시', '봐도', '재밌', '어요', 'ㅋㅋ'], ['디자인', '을', '배우', '는', '학생', '으로', ',', '외국', '디자이너', '와', '그', '들', '이', '일군', '전통', '을', '통해', '발전', '해', '가', '는', '문화', '산업', '이', '부러웠', '는데', '.', '사실', '우리', '나라', '에서', '도', '그', '어려운', '시절', '에', '끝', '까지', '열정', '을', '지킨', '노라노', '같', '은', '전통', '이', '있', '어', '저', '와', '같', '은', '사람', '들', '이', '꿈', '을', '꾸', '고', '이뤄나갈', '수', '있', '다는', '것', '에', '감사', '합니다', '.'], ['폴리스', '스토리', '시리즈', '는', '1', '부터', '뉴', '까지', '버릴', '께', '하나', '도', '없', '음', '.', '.', '최고', '.']]
199993
Wall time: 19.3 s


# Train Tokenizer

downstream task를 위해 UNK와 많은 unused가 필요하다.

1) define special tokens

2) train

In [3]:
## 1) define special tokens
user_defined_symbols = ['[BOS]','[EOS]','[UNK0]','[UNK1]','[UNK2]','[UNK3]','[UNK4]','[UNK5]','[UNK6]','[UNK7]','[UNK8]','[UNK9]']
unused_token_num = 200
unused_list = ['[unused{}]'.format(n) for n in range(unused_token_num)]
user_defined_symbols = user_defined_symbols + unused_list

print(user_defined_symbols)

['[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]', '[unused0]', '[unused1]', '[unused2]', '[unused3]', '[unused4]', '[unused5]', '[unused6]', '[unused7]', '[unused8]', '[unused9]', '[unused10]', '[unused11]', '[unused12]', '[unused13]', '[unused14]', '[unused15]', '[unused16]', '[unused17]', '[unused18]', '[unused19]', '[unused20]', '[unused21]', '[unused22]', '[unused23]', '[unused24]', '[unused25]', '[unused26]', '[unused27]', '[unused28]', '[unused29]', '[unused30]', '[unused31]', '[unused32]', '[unused33]', '[unused34]', '[unused35]', '[unused36]', '[unused37]', '[unused38]', '[unused39]', '[unused40]', '[unused41]', '[unused42]', '[unused43]', '[unused44]', '[unused45]', '[unused46]', '[unused47]', '[unused48]', '[unused49]', '[unused50]', '[unused51]', '[unused52]', '[unused53]', '[unused54]', '[unused55]', '[unused56]', '[unused57]', '[unused58]', '[unused59]', '[unused60]', '[unused61]', '[unused62]', '[unused6

In [4]:
%%time
## 2) train
import os
from tokenizers import BertWordPieceTokenizer, SentencePieceBPETokenizer, CharBPETokenizer, ByteLevelBPETokenizer

# 4가지중 tokenizer 선택
how_to_tokenize = BertWordPieceTokenizer  # The famous Bert tokenizer, using WordPiece
# how_to_tokenize = SentencePieceBPETokenizer  # A BPE implementation compatible with the one used by SentencePiece
# how_to_tokenize = CharBPETokenizer  # The original BPE
# how_to_tokenize = ByteLevelBPETokenizer  # The byte level version of the BPE

# Initialize a tokenizer
if str(how_to_tokenize) == str(BertWordPieceTokenizer):
    print('BertWordPieceTokenizer')
    ## 주의!! 한국어는 strip_accents를 False로 해줘야 한다
    # 만약 True일 시 나는 -> 'ㄴ','ㅏ','ㄴ','ㅡ','ㄴ' 로 쪼개져서 처리된다
    # 학습시 False했으므로 load할 때도 False를 꼭 확인해야 한다
    tokenizer = BertWordPieceTokenizer(strip_accents=False,  # Must be False if cased model
                                       lowercase=False)
elif str(how_to_tokenize) == str(SentencePieceBPETokenizer):
    print('SentencePieceBPETokenizer')
    tokenizer = SentencePieceBPETokenizer()

elif str(how_to_tokenize) == str(CharBPETokenizer):
    print('CharBPETokenizer')
    tokenizer = CharBPETokenizer()
    
elif str(how_to_tokenize) == str(ByteLevelBPETokenizer):
    print('ByteLevelBPETokenizer')
    tokenizer = ByteLevelBPETokenizer()
       
else:
    assert('select right tokenizer')

#########################################
corpus_file   = ['data/after_mecab.txt']  # data path
vocab_size    = 32000
limit_alphabet= 6000
output_path   = 'hugging_%d'%(vocab_size)
min_frequency = 5

# Then train it!
tokenizer.train(files=corpus_file,
               vocab_size=vocab_size,
               min_frequency=min_frequency,  # 단어의 최소 발생 빈도, 5
               limit_alphabet=limit_alphabet,  # ByteLevelBPETokenizer 학습시엔 주석처리 필요
               show_progress=True)
print('train complete')

sentence = '나는 오늘 아침밥을 먹었다.'
output = tokenizer.encode(sentence)
print(sentence)
print('=>tokens: %s'%output.tokens)
print('=>idx   : %s'%output.ids)
print('=>offset: %s'%output.offsets)
print('=>decode: %s\n'%tokenizer.decode(output.ids))

sentence = 'I want to go my hometown'
output = tokenizer.encode(sentence)
print(sentence)
print('=>tokens: %s'%output.tokens)
print('=>idx   : %s'%output.ids)
print('=>offset: %s'%output.offsets)
print('=>decode: %s\n'%tokenizer.decode(output.ids))

# save tokenizer
hf_model_path='tokenizer_model'
if not os.path.isdir(hf_model_path):
    os.mkdir(hf_model_path)
tokenizer.save_model(hf_model_path)  # vocab.txt 파일 한개가 만들어진다

BertWordPieceTokenizer
train complete
나는 오늘 아침밥을 먹었다.
=>tokens: ['나', '##는', '오늘', '아침', '##밥', '##을', '먹', '##었다', '.']
=>idx   : [875, 3288, 5446, 6142, 3380, 3653, 1474, 17171, 18]
=>offset: [(0, 1), (1, 2), (3, 5), (6, 8), (8, 9), (9, 10), (11, 12), (12, 14), (14, 15)]
=>decode: 나는 오늘 아침밥을 먹었다.

I want to go my hometown
=>tokens: ['I', 'want', 'to', 'go', 'my', 'h', '##ome', '##t', '##own']
=>idx   : [45, 17424, 7701, 11757, 10072, 76, 11902, 3315, 22586]
=>offset: [(0, 1), (2, 6), (7, 9), (10, 12), (13, 15), (16, 17), (17, 20), (20, 21), (21, 24)]
=>decode: I want to go my hometown

Wall time: 6.93 s


['tokenizer_model\\vocab.txt']

# Check import tokenizer from Transformers

BertTokenizerFast 는 학습한 환경과 똑같이 strip_accents=False 로 줘야한다

True로 주면 '나는'->'ㄴㅏㄴㅡㄴ' 과 같이 쪼개져서 인식해서 안된다

In [5]:
from transformers import BertTokenizerFast

tokenizer_for_load = BertTokenizerFast.from_pretrained(hf_model_path,
                                                       strip_accents=False,  # Must be False if cased model
                                                       lowercase=False)  # 로드

print('vocab size : %d' % tokenizer_for_load.vocab_size)
# tokenized_input_for_pytorch = tokenizer_for_load("i am very hungry", return_tensors="pt")
tokenized_input_for_pytorch = tokenizer_for_load("나는 오늘 아침밥을 먹었다.", return_tensors="pt")
tokenized_input_for_tensorflow = tokenizer_for_load("나는 오늘 아침밥을 먹었다.", return_tensors="tf")

print("Tokens (str)      : {}".format([tokenizer_for_load.convert_ids_to_tokens(s) for s in tokenized_input_for_pytorch['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(tokenized_input_for_pytorch['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(tokenized_input_for_pytorch['attention_mask'].tolist()[0]))

vocab size : 27305
Tokens (str)      : ['[CLS]', '나', '##는', '오늘', '아침', '##밥', '##을', '먹', '##었다', '.', '[SEP]']
Tokens (int)      : [2, 875, 3288, 5446, 6142, 3380, 3653, 1474, 17171, 18, 3]
Tokens (attn_mask): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]



In [6]:
# vocab check
tokenizer_for_load.get_vocab()

{'##わ': 4236,
 '2013': 8497,
 '머시': 25259,
 '##저릿': 24079,
 '해짐': 8888,
 '확확': 22252,
 '권리': 10850,
 '도저': 5773,
 'ㅇㅋㅋ': 24854,
 '인생': 5224,
 '베껴': 13087,
 '딜레마': 15600,
 '뢔': 1374,
 '##베이터': 13207,
 '뽑아낸': 27018,
 '##난다': 5778,
 '저능아': 12105,
 '남겨요': 16591,
 '셸': 1898,
 '쉣': 1950,
 '바늘': 17592,
 '올려': 6459,
 '##하하': 5934,
 '욕정': 17704,
 '##제네거': 18754,
 '괜찮': 5212,
 '일수록': 17739,
 'CD': 16722,
 '류헤이': 25218,
 '아파지': 26667,
 '피아': 7408,
 '지상': 7895,
 '연기': 5117,
 '텐': 2891,
 '인간': 5248,
 '뒤엎': 18297,
 'have': 20143,
 '##스크': 7873,
 '사골': 11627,
 '쨩': 2556,
 '아델': 21859,
 '이완용': 26824,
 '덱스터': 17532,
 '드만': 8616,
 '부': 1682,
 '컴퓨터': 7215,
 '워낭': 15825,
 '뮬란': 14390,
 '재주': 9329,
 '률': 1403,
 '그땐': 11421,
 '×': 109,
 '옆집': 13136,
 '웟다': 14849,
 '잡쳐': 25843,
 '엉터리': 9008,
 '히말라야': 21341,
 '##꼰': 5038,
 '##수정': 8598,
 '亡': 336,
 '백마': 23340,
 '발칙': 12599,
 '개욕': 24890,
 '커져': 23845,
 '자신': 5423,
 '엘리': 7687,
 '던진다': 21571,
 '교정': 24946,
 '졌': 2423,
 '##크ㅋ': 22294,
 '픽픽': 23942,
 '졸려': 8722

In [7]:
# special token check
tokenizer_for_load.all_special_tokens # 추가하기 전 기본적인 special token

['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

In [8]:
# tokenizer에 special token 추가
special_tokens_dict = {'additional_special_tokens': user_defined_symbols}
tokenizer_for_load.add_special_tokens(special_tokens_dict)

# check tokenizer vocab with special tokens
print('check special tokens : %s'%tokenizer_for_load.all_special_tokens[:20])

check special tokens : ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]', '[unused0]', '[unused1]', '[unused2]']


In [9]:
# save tokenizer model with special tokens
tokenizer_for_load.save_pretrained(hf_model_path+'_special')

('tokenizer_model_special\\tokenizer_config.json',
 'tokenizer_model_special\\special_tokens_map.json',
 'tokenizer_model_special\\vocab.txt',
 'tokenizer_model_special\\added_tokens.json')

In [10]:
# check special tokens
from transformers import BertTokenizerFast
tokenizer_check = BertTokenizerFast.from_pretrained(hf_model_path+'_special')

print('check special tokens : %s'%tokenizer_check.all_special_tokens[:20])

print('vocab size : %d' % tokenizer_check.vocab_size)
tokenized_input_for_pytorch = tokenizer_check("나는 오늘 아침밥을 먹었다.", return_tensors="pt")
tokenized_input_for_tensorflow = tokenizer_check("나는 오늘 아침밥을 먹었다.", return_tensors="tf")

print("Tokens (str)      : {}".format([tokenizer_check.convert_ids_to_tokens(s) for s in tokenized_input_for_pytorch['input_ids'].tolist()[0]]))
print("Tokens (int)      : {}".format(tokenized_input_for_pytorch['input_ids'].tolist()[0]))
print("Tokens (attn_mask): {}\n".format(tokenized_input_for_pytorch['attention_mask'].tolist()[0]))

check special tokens : ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]', '[BOS]', '[EOS]', '[UNK0]', '[UNK1]', '[UNK2]', '[UNK3]', '[UNK4]', '[UNK5]', '[UNK6]', '[UNK7]', '[UNK8]', '[UNK9]', '[unused0]', '[unused1]', '[unused2]']
vocab size : 27305
Tokens (str)      : ['[CLS]', '나', '##는', '오늘', '아침', '##밥', '##을', '먹', '##었다', '.', '[SEP]']
Tokens (int)      : [2, 875, 3288, 5446, 6142, 3380, 3653, 1474, 17171, 18, 3]
Tokens (attn_mask): [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]



In [11]:
# test to tf&pytorch bert model
from transformers import TFBertModel, BertModel

# load a BERT model for TensorFlow and PyTorch
model_tf = TFBertModel.from_pretrained('bert-base-cased')
model_pt = BertModel.from_pretrained('bert-base-cased')

Some layers from the model checkpoint at bert-base-cased were not used when initializing TFBertModel: ['mlm___cls', 'nsp___cls']
- This IS expected if you are initializing TFBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
All the layers of TFBertModel were initialized from the model checkpoint at bert-base-cased.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions without further training.


In [12]:
## tf vs torch bert output
# transformers generates a ready to use dictionary with all the required parameters for the specific framework.
input_tf = tokenizer_check("나는 오늘 아침밥을 먹었다.", return_tensors="tf")
input_pt = tokenizer_check("나는 오늘 아침밥을 먹었다.", return_tensors="pt")

# Let's compare the outputs
output_tf, output_pt = model_tf(input_tf), model_pt(**input_pt)

print('final layer output shape : %s'%(output_pt['last_hidden_state'].shape,))

# Models outputs 2 values (The value for each tokens, the pooled representation of the input sentence)
# Here we compare the output differences between PyTorch and TensorFlow.

print('\ntorch vs tf 결과차이')
for name in ["last_hidden_state", "pooler_output"]:
    print("   => {} differences: {:.5}".format(name, (output_tf[name].numpy() - output_pt[name].detach().numpy()).sum()))

final layer output shape : torch.Size([1, 11, 768])

torch vs tf 결과차이
   => last_hidden_state differences: 1.1779e-06
   => pooler_output differences: 6.1418e-06
